# PSV Curves

Here we will show how to calculate paleosecular variation curves with `pypsv`. We start by setting up the input data:

In [1]:
import numpy as np
from pandas import DataFrame

input_data = DataFrame(
    data={
        'F': [40, 45],
        'dF': [1.25, 0.7],
        'Age': [879., 1595.],
        'dAge': [20, np.nan],
        'Age type': ["14C SH", "absolute"],
    },
)

As discussed in a different example, this dataset contains two intensity records. One is radiocarbon dated and the other is absolutely dated. We assume this data stems from Fiji, so we define the location tuple accordingly:

In [2]:
# Fiji latitude and longitude
loc = (-18.166667, 178.45)

Next, we have to define the desired duration and resolution of the PSV curve as a `numpy` array. We want the curve to extend from 1000 CE to 2000 CE with knots every 20 years. Due to the way `arange` works in `numpy`, the upper limit is set to 2001, so that the endpoint is included.

In [3]:
curve_knots = np.arange(1000, 2001, 20)

Now we have assembled all necessary inputs to set up a `PSVCurve` object:

In [4]:
from pypsv import PSVCurve

curve = PSVCurve(
    loc=loc,
    curve_knots=curve_knots,
    data=input_data,
)

We can generate an ensemble of PSV curves by sampling as follows:

In [5]:
idata = curve.sample()

Setting up PyMC model...
...model setup done.


  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details


The resulting `InferenceData` object can be exported as a `netCDF` file. We show how to work with this output in the next example.

In [6]:
idata.to_netcdf(
    './example_out.nc',
)

'./example_out.nc'

### Using field models as prior

To include knowledge from the global geomagnetic field in the PSV Curve, we can use a field model as prior. To do this, a new `PSVCurve` instance has to be created with the desired field model prior. First, the field model has to be imported:

In [7]:
from pypsv.fieldmodels import ArchKalmag14k

This can then be passed to the `PSVCurve` class:

In [8]:
akm_curve = PSVCurve(
    loc=loc,
    curve_knots=curve_knots,
    data=input_data,
    prior_model=ArchKalmag14k(),
)

A new ensemble can be generated and stored, similar as before:

In [9]:
akm_idata = akm_curve.sample()
akm_idata.to_netcdf(
    './example_out_akm.nc',
)

Setting up PyMC model...
...model setup done.


  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

'./example_out_akm.nc'